In [32]:
import pandas as pd
from geolite2 import geolite2
import re

## 1) Извлечь как можно больше информации из ip-адреса и user agent-а и записать в отдельный csv файл

In [33]:
data = pd.read_csv('group-ib-test.csv', sep=";", header=0)

###### Подготовливаем ip адреса для проверки  через geolite2

In [34]:
data_for_geolite = data.ip.str.replace(r"['\s([)\&\]]","")

In [35]:
data_for_geolite = data_for_geolite.str.replace(r",.*","")

###### Убираем предидущее столбец с ip и объединяем данные

In [36]:
data.drop("ip", axis=1, inplace=True)

In [37]:
data_for_geolite = pd.concat((data_for_geolite, data), axis=1)

###### Создаем функцию для проверки и проверяем ip адреса

In [38]:
def f(item):
    match = geolite2.reader()
    return match.get(item)

In [39]:
data_from_geolite = data_for_geolite.ip.apply(f)

In [40]:
data_from_geolite = pd.concat((data_from_geolite, data_for_geolite), axis=1)

In [41]:
labels = ["ip_info", "ip", "device_id", "login","user_agent"]

In [42]:
data_from_geolite.columns = labels

###### Извлекаем данные из user_agent с помощью регулярных выражений и сохраняем файл

In [43]:
data_user_agent = data_from_geolite["user_agent"]

In [44]:
data_os = data_user_agent.replace("^Mozilla/5.0\s\(|\).*", "", regex=True)

In [45]:
data_mob = data_user_agent.str.contains(".obile", regex=True)

In [50]:
re_browsers = "(\S*$)|(Chr.*)"

In [51]:
data_browsers = data_user_agent.str.extract(re_browsers)

/Users/dvpisarev/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
  """Entry point for launching an IPython kernel.


In [52]:
data_browsers = data_browsers[0].fillna(data_browsers[1])

In [53]:
data_to_csv = pd.concat((data_from_geolite, data_os, data_browsers, data_mob ), axis=1)

In [57]:
labels2 = ["ip_info", "ip", "device_id", "login","user_agent", "os", "browser", "mobile"]

In [58]:
data_to_csv.columns = labels2

In [56]:
data_to_csv.to_csv("test-grp-ib.csv")

# 2) Показать, какие идентификаторы устройства принадлежат одному и тому же реальному устройству и почему?

###### Создаем датафрейм с идентификаторами и информации из user_agent и группируем

In [68]:
data_devid_usagent = data_to_csv[["user_agent", "device_id"]]

In [69]:
for _, x in data_devid_usagent.groupby(["user_agent"]): print (60*"-","\n\n", x)

------------------------------------------------------------ 

                                              user_agent  \
1981  Mozilla/5.0 (Linux, Android 5.0, ASUS_T00J Bui...   

                                 device_id  
1981  29501893-72c0-43e8-826a-9d5673c1aef4  
------------------------------------------------------------ 

                                              user_agent  \
1979  Mozilla/5.0 (Linux, Android 5.0, ASUS_T00J Bui...   
1980  Mozilla/5.0 (Linux, Android 5.0, ASUS_T00J Bui...   

                                 device_id  
1979  29501893-72c0-43e8-826a-9d5673c1aef4  
1980  29501893-72c0-43e8-826a-9d5673c1aef4  
------------------------------------------------------------ 

                                              user_agent  \
1970  Mozilla/5.0 (Linux, Android 5.0, ASUS_T00J Bui...   
1971  Mozilla/5.0 (Linux, Android 5.0, ASUS_T00J Bui...   
1972  Mozilla/5.0 (Linux, Android 5.0, ASUS_T00J Bui...   
1973  Mozilla/5.0 (Linux, Android 5.0, ASUS_T00J B

                                              user_agent  \
1234  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1235  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1236  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1237  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1238  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1239  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1240  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1241  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1242  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1243  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1244  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1245  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1246  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1247  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1248  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...   
1249  Mozilla/5.0 (Linux, Android 7.1.2, Swift 2 Plu...

                                              user_agent  \
1743  Mozilla/5.0 (Windows NT 10.0, Win64, x64, rv:6...   

                                 device_id  
1743  9ba80e4f-c8af-4d59-a4f8-88b53f8fbfe9  
------------------------------------------------------------ 

                                             user_agent  \
857  Mozilla/5.0 (Windows NT 10.0, Win64, x64, rv:6...   

                                device_id  
857  42c929d2-2b6c-40c8-b588-5f079ba3fe7d  
------------------------------------------------------------ 

                                              user_agent  \
2197  Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...   
2198  Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...   
2199  Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...   
2200  Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...   
2201  Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...   
2230  Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.3...   
2231  Mozilla/5.0 (Windows NT 6.1) AppleWebK

                                              user_agent  \
1685  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1686  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1687  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1688  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1689  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1768  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1769  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1770  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1771  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1772  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1773  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1774  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1775  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1776  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1777  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...   
1778  Mozilla/5.0 (iPhone, CPU iPhone OS 11_4 like M...

# 3) Найти связи между устройствами и логинами

###### Создаем датафрейм с нужными колонками и группируем по логину

In [61]:
data_login = data_to_csv[["login","device_id",  "os", "browser"]]

In [74]:
for _, x in data_login.groupby(["login"]): print (60*"-","\n\n", x)

------------------------------------------------------------ 

                                          login  \
1628  0215ba87c2e630e69a7abc5efcafc3cd3ede9aec   
1629  0215ba87c2e630e69a7abc5efcafc3cd3ede9aec   
1630  0215ba87c2e630e69a7abc5efcafc3cd3ede9aec   
1631  0215ba87c2e630e69a7abc5efcafc3cd3ede9aec   
1632  0215ba87c2e630e69a7abc5efcafc3cd3ede9aec   
1633  0215ba87c2e630e69a7abc5efcafc3cd3ede9aec   
1634  0215ba87c2e630e69a7abc5efcafc3cd3ede9aec   

                                 device_id  \
1628  ff5be60e-b8c9-4158-8d02-24b4075002bb   
1629  ff5be60e-b8c9-4158-8d02-24b4075002bb   
1630  9d31760b-33db-4f19-88a5-872f9dc3bcc4   
1631  9d31760b-33db-4f19-88a5-872f9dc3bcc4   
1632  9d31760b-33db-4f19-88a5-872f9dc3bcc4   
1633  9d31760b-33db-4f19-88a5-872f9dc3bcc4   
1634  9d31760b-33db-4f19-88a5-872f9dc3bcc4   

                                                     os  \
1628  Linux, Android 8.0.0, ANE-LX1 Build/HUAWEIANE-...   
1629  Linux, Android 8.0.0, ANE-LX1 Build/HUAWEI

89  Chrome/66.0.3359.139 Safari/537.36  
------------------------------------------------------------ 

                                         login  \
315  111b1935f6ca4b46b6613cedf65a4a59a9533d82   
324  111b1935f6ca4b46b6613cedf65a4a59a9533d82   

                                device_id  \
315  7157122e-8200-472d-893c-95b10010ce38   
324  7157122e-8200-472d-893c-95b10010ce38   

                                      os       browser  
315  Windows NT 6.1, Win64, x64, rv:59.0  Firefox/59.0  
324  Windows NT 6.1, Win64, x64, rv:59.0  Firefox/59.0  
------------------------------------------------------------ 

                                          login  \
1330  1184aea39ce405be183c29ee20b2cdffb1463007   
1340  1184aea39ce405be183c29ee20b2cdffb1463007   

                                 device_id                                 os  \
1330  e9ded28b-43b0-415c-80bc-5a332ba1573c  Macintosh, Intel Mac OS X 10_13_4   
1340  e9ded28b-43b0-415c-80bc-5a332ba1573c  Macintosh, Intel Ma

                                         login  \
873  1e9e39996ab0ddad6bf0fe431abe693f55426349   
882  1e9e39996ab0ddad6bf0fe431abe693f55426349   

                                device_id                          os  \
873  15925cb9-fefe-413c-be86-bf0650b2d2df  Windows NT 6.1, Win64, x64   
882  15925cb9-fefe-413c-be86-bf0650b2d2df  Windows NT 6.1, Win64, x64   

                                browser  
873   Chrome/67.0.3396.87 Safari/537.36  
882  Chrome/66.0.3359.139 Safari/537.36  
------------------------------------------------------------ 

                                          login  \
2195  1fa2a9995815c61b03b4537d4904a85d87ac31fb   
2200  1fa2a9995815c61b03b4537d4904a85d87ac31fb   
2207  1fa2a9995815c61b03b4537d4904a85d87ac31fb   
2211  1fa2a9995815c61b03b4537d4904a85d87ac31fb   
2213  1fa2a9995815c61b03b4537d4904a85d87ac31fb   

                                 device_id                     os  \
2195  886ee7bb-2c79-4a17-924b-9383fe982e91         Windows NT 6.1   
22

                                         login  \
5    3062ace3cea5074e292b2e9413b4172390dc99ea   
343  3062ace3cea5074e292b2e9413b4172390dc99ea   
522  3062ace3cea5074e292b2e9413b4172390dc99ea   
627  3062ace3cea5074e292b2e9413b4172390dc99ea   

                                device_id  \
5    1a337b50-d8c7-49bd-8460-949f758f05ab   
343  75abe5ab-7944-45d1-8271-08407847627e   
522  cb2aa02f-fb5c-4404-8122-916e223fdad3   
627  d1a791ab-0d54-4d28-8199-eae1ff5d7c83   

                                      os                            browser  
5            Windows NT 10.0, Win64, x64  Chrome/67.0.3396.99 Safari/537.36  
343  Windows NT 6.1, Win64, x64, rv:61.0                       Firefox/61.0  
522          Windows NT 10.0, Win64, x64  Chrome/67.0.3396.99 Safari/537.36  
627           Windows NT 6.1, Win64, x64  Chrome/67.0.3396.99 Safari/537.36  
------------------------------------------------------------ 

                                         login  \
895  30c66e38bd59562f548

                                          login  \
1482  42d7eb866e39ee159a03b34a59d9758663f33c12   
1483  42d7eb866e39ee159a03b34a59d9758663f33c12   
1484  42d7eb866e39ee159a03b34a59d9758663f33c12   
1485  42d7eb866e39ee159a03b34a59d9758663f33c12   
1486  42d7eb866e39ee159a03b34a59d9758663f33c12   
1487  42d7eb866e39ee159a03b34a59d9758663f33c12   
1488  42d7eb866e39ee159a03b34a59d9758663f33c12   
1489  42d7eb866e39ee159a03b34a59d9758663f33c12   
1490  42d7eb866e39ee159a03b34a59d9758663f33c12   
1491  42d7eb866e39ee159a03b34a59d9758663f33c12   
1492  42d7eb866e39ee159a03b34a59d9758663f33c12   
1493  42d7eb866e39ee159a03b34a59d9758663f33c12   
1494  42d7eb866e39ee159a03b34a59d9758663f33c12   
1495  42d7eb866e39ee159a03b34a59d9758663f33c12   
1496  42d7eb866e39ee159a03b34a59d9758663f33c12   
1497  42d7eb866e39ee159a03b34a59d9758663f33c12   
1498  42d7eb866e39ee159a03b34a59d9758663f33c12   
1499  42d7eb866e39ee159a03b34a59d9758663f33c12   
1500  42d7eb866e39ee159a03b34a59d9758663f33c12   

...

                                         login  \
31   81ac546603fd7563a77b897c49d1c42171f163c8   
596  81ac546603fd7563a77b897c49d1c42171f163c8   

                                device_id                           os  \
31   1a337b50-d8c7-49bd-8460-949f758f05ab  Windows NT 10.0, Win64, x64   
596  cb2aa02f-fb5c-4404-8122-916e223fdad3  Windows NT 10.0, Win64, x64   

                                browser  
31   Chrome/66.0.3359.181 Safari/537.36  
596  Chrome/66.0.3359.181 Safari/537.36  
------------------------------------------------------------ 

                                         login  \
291  8249824832a67ac92d7e3ae43fafd81ddc3faf1b   
391  8249824832a67ac92d7e3ae43fafd81ddc3faf1b   

                                device_id  \
291  7157122e-8200-472d-893c-95b10010ce38   
391  75abe5ab-7944-45d1-8271-08407847627e   

                                      os       browser  
291  Windows NT 6.1, Win64, x64, rv:59.0  Firefox/59.0  
391  Windows NT 6.1, Win64, x64, rv:59.0

                                         login  \
543  9674f8bc2e41218f497645e5cfd9c2b916152690   
566  9674f8bc2e41218f497645e5cfd9c2b916152690   

                                device_id                           os  \
543  cb2aa02f-fb5c-4404-8122-916e223fdad3  Windows NT 10.0, Win64, x64   
566  cb2aa02f-fb5c-4404-8122-916e223fdad3  Windows NT 10.0, Win64, x64   

                               browser  
543  Chrome/67.0.3396.99 Safari/537.36  
566  Chrome/67.0.3396.99 Safari/537.36  
------------------------------------------------------------ 

                                         login  \
495  9683d704698c60eeb993f80b0274b754c68cffd4   

                                device_id                     os  \
495  b5251a99-f6db-42b8-9ab8-9a216a3119bf  Windows NT 6.1, WOW64   

                              browser  
495  Chrome/53.0.2764.0 Safari/537.36  
------------------------------------------------------------ 

                                         login  \
320  9698e90

654  Chrome/67.0.3396.99 Safari/537.36  
------------------------------------------------------------ 

                                         login  \
544  ae33f551d547431f5f7f05ff56741d409b4e1cac   
662  ae33f551d547431f5f7f05ff56741d409b4e1cac   
667  ae33f551d547431f5f7f05ff56741d409b4e1cac   

                                device_id                           os  \
544  cb2aa02f-fb5c-4404-8122-916e223fdad3  Windows NT 10.0, Win64, x64   
662  d1a791ab-0d54-4d28-8199-eae1ff5d7c83   Windows NT 6.1, Win64, x64   
667  d1a791ab-0d54-4d28-8199-eae1ff5d7c83   Windows NT 6.1, Win64, x64   

                               browser  
544  Chrome/67.0.3396.99 Safari/537.36  
662  Chrome/67.0.3396.99 Safari/537.36  
667  Chrome/67.0.3396.99 Safari/537.36  
------------------------------------------------------------ 

                                         login  \
130  ae717d2cfd5439b1b312c394c0d4dc72a4c7695b   
634  ae717d2cfd5439b1b312c394c0d4dc72a4c7695b   

                         

                                        login  \
50  cd2dd90ea6cb1d4c458ada92a51f71d10d4877a6   
56  cd2dd90ea6cb1d4c458ada92a51f71d10d4877a6   
88  cd2dd90ea6cb1d4c458ada92a51f71d10d4877a6   
93  cd2dd90ea6cb1d4c458ada92a51f71d10d4877a6   
96  cd2dd90ea6cb1d4c458ada92a51f71d10d4877a6   

                               device_id                          os  \
50  26023b66-e3f5-4496-884b-e881adb2d793  Windows NT 6.1, Win64, x64   
56  26023b66-e3f5-4496-884b-e881adb2d793  Windows NT 6.1, Win64, x64   
88  26023b66-e3f5-4496-884b-e881adb2d793  Windows NT 6.1, Win64, x64   
93  26023b66-e3f5-4496-884b-e881adb2d793  Windows NT 6.1, Win64, x64   
96  26023b66-e3f5-4496-884b-e881adb2d793  Windows NT 6.1, Win64, x64   

                               browser  
50   Chrome/67.0.3396.99 Safari/537.36  
56   Chrome/67.0.3396.87 Safari/537.36  
88  Chrome/66.0.3359.139 Safari/537.36  
93  Chrome/66.0.3359.139 Safari/537.36  
96  Chrome/66.0.3359.139 Safari/537.36  
-------------------------------

1323  Chrome/65.0.3325.181 Safari/537.36  
------------------------------------------------------------ 

                                          login  \
2176  ed72e5835fd51a338a4b574f47b150338197f2a7   
2178  ed72e5835fd51a338a4b574f47b150338197f2a7   

                                 device_id  \
2176  25f5539d-9f43-4199-b0dd-3e604d4c27be   
2178  a156818e-d8c2-4892-96d3-4e938f81b909   

                                                os browser  
2176  Windows NT 10.0, WOW64, Trident/7.0, rv:11.0   Gecko  
2178   Windows NT 6.1, WOW64, Trident/7.0, rv:11.0   Gecko  
------------------------------------------------------------ 

                                          login  \
1123  ed9b3e652da241191a4d2f52d6c89d84295a49d3   
1124  ed9b3e652da241191a4d2f52d6c89d84295a49d3   
1125  ed9b3e652da241191a4d2f52d6c89d84295a49d3   
1126  ed9b3e652da241191a4d2f52d6c89d84295a49d3   
1127  ed9b3e652da241191a4d2f52d6c89d84295a49d3   
1128  ed9b3e652da241191a4d2f52d6c89d84295a49d3   
1129  

###### Просматриваем наиболее используемый логин и устройства на которых он использовался

In [75]:
top_logins = data_to_csv["login"].value_counts()

In [81]:
top_logins.head(10)

ed9b3e652da241191a4d2f52d6c89d84295a49d3    234
d21e16e4f73eee03a4af84277fa2d2c7450842f6     70
5071432df9016a009c99246a0c818d40615ef9e6     66
2d3471dcd49bb9964344200ad733b995aecac0aa     49
273e6429a6f100c3cd628f9381ddd1fce46eb922     47
bf07c1d8685590e1e4128d115970ad372febf630     47
4ef7dfd0aa55f412fd235edc2ef39c414430e8ef     45
42d7eb866e39ee159a03b34a59d9758663f33c12     41
5990488be330738b5e253883af8922a394c52ca4     36
e16edfc7ccd57ea881487fd1f138a2e8c61ab012     36
Name: login, dtype: int64

In [82]:
the_top_login = data_to_csv[data_to_csv['login'] == "ed9b3e652da241191a4d2f52d6c89d84295a49d3"]

In [83]:
the_top_login["user_agent"].value_counts()

Mozilla/5.0 (Linux, Android 5.1, m2 Build/LMY47D, wv) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/44.0.2403.147 Mobile Safari/537.36    231
Mozilla/5.0 (Windows NT 10.0, WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 YaBrowser/18.2.1.196 Yowser/2.5 Safari/537.36          3
Name: user_agent, dtype: int64